## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:

import scipy.signal
import glob

import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

%matplotlib inline

minfreq = 40/60.0
maxfreq = 240/60.0

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    print("mean error", errs.mean())
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    The algorithm uses the Power Spectral Density(PSD) estimates to select frequencies 
    with high PSDs for both PPG and Accelerometer signals. It then remove the frequencies 
    which could have been attributed by accelerometer by comaring the PPG frequency to 
    that of top three frequencies of accelerometer. 
    
    Args: 
        data_fl: (str) filepath to a troika .mat file.
        ref_f1: (str) filepath to a troika .mat file which contains reference heart beat estimates.
        
    Returns:
        error_array: Array of absolute error.
        conf_array: Array of confidence values for each erroe value.
    """
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # loading the reference file
    ground_truth = np.array(sp.io.loadmat(ref_fl)['BPM0'])
    ground_truth = ground_truth.reshape(len(ground_truth))
    
    # Bandpass ppg and acc signals 
    ppg = bandpass_filter(ppg)
    accx = bandpass_filter(accx)
    accy = bandpass_filter(accy)
    accz = bandpass_filter(accz)
        
    # Calculate Power Spectral Density estimates for each signal
    psd_ppg, freqs_ppg = cal_spectogram(signal=ppg)
    psd_accx, freqs_accx = cal_spectogram(signal=accx)
    psd_accy, freqs_accy = cal_spectogram(signal=accy)
    psd_accz, freqs_accz = cal_spectogram(signal=accz)
    
    
    time_steps = psd_ppg.shape[1]
    freqs = freqs_ppg.shape[0]
    
    # Sorted index value for signals 
    ppg_index = (-psd_ppg).argsort(axis=0)
    accx_index = (-psd_accx).argsort(axis=0)
    accy_index = (-psd_accy).argsort(axis=0)
    accz_index = (-psd_accz).argsort(axis=0)

    estimated_freq = []
    for t in range(time_steps):
        for freq in range(freqs):
            
            idx=0
            # compare frequency values of ppg withat of acc siganls
            # pick only those ppg frequencies which are not present in the top 3 frequencies of acc signals 
            if freq == 2:
                estimated_freq.append(freqs_ppg[ppg_index[freq][t]])
                break
            elif np.all([(freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[idx][t]]), 
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[idx][t]]), 
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[idx][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[idx+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[idx+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[idx+1][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accx_index[idx+2][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accy_index[idx+2][t]]),
                      (freqs_ppg[ppg_index[freq][t]] != freqs_ppg[accz_index[idx+2][t]])]):
                estimated_freq.append(freqs_ppg[ppg_index[freq][t]])
                break
            
    # claculate confidence using snr
    confidence = []
    for idx in range(len(estimated_freq)):
        snr = calc_snr(ppg, estimated_freq[idx])
        confidence.append(snr)
            
    pre = np.array(estimated_freq) * 60
    
    # absolute error 
    error_array = np.abs(ground_truth - pre)
    
    conf_array = np.array(confidence)


    # Return per-estimate absolute error and confidence as a 2-tuple of numpy arrays.

    return error_array, conf_array


def bandpass_filter(signal):
    """
    Bandpass filter the signal between 40 and 240 BPM
    
    Args: 
        signal: signal from PPG or Accelerometer
    
    Returns:
        Band pass filtered signal.
  
    """    
    pass_band=(40/60.0, 240/60.0)
    fs = 125
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def calc_snr(signal, hr_f):
    """
    Calculate signal to noise ratio
    
    Args:
        signal: signal from PPG
        hr_f: heart rate frequency estimates
        
    Returns:
        snr: signal to noise ratio
    
    """
    n = len(signal)*2
    fs = 125
    
    #calculate the frequency of the first harmonic
    harmonic_f = hr_f * 2
    
    #compute the fft
    fft_mag = np.abs(np.fft.rfft(signal, n))
    freqs = np.fft.rfftfreq(n, 1/fs)
    
    #find the frequencies that are around the heart rate and the first harmonic of the heart rate
    window_f = 5/60
    
    fundamental_frequency_window = (freqs > hr_f - window_f) & (freqs < hr_f + window_f)
    harmonic_frequency_window = (freqs > harmonic_f - window_f) & (freqs < harmonic_f + window_f)
    
    signal_power = np.sum(fft_mag[(fundamental_frequency_window) | (harmonic_frequency_window)])
    noise_power = np.sum(fft_mag[-((fundamental_frequency_window) | (harmonic_frequency_window))])
    
    snr = signal_power / noise_power
    
    return snr
    
    

def cal_spectogram(signal):
    """
    Calculate Power Spectral Density estimates
    
    Args:
        signal: PPG or Accelerometer signals
    Returns:
        psd: Power spectral density estimate
        freqs:  frequencies for which PSDs are calculated
    
    """
    fs = 125
    psd, freqs, t = mlab.specgram(signal, NFFT=8*fs, Fs=fs, noverlap=6*fs, pad_to=12*fs)
    psd = psd[(freqs >= minfreq) & (freqs <= maxfreq)]
    freqs = freqs[(freqs >= minfreq) & (freqs <= maxfreq)]
    return psd, freqs
    

In [2]:
Evaluate()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:256: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


mean error 19.6074384408


19.499553789676362

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

###### Code Description
The code can be run by executing Evaluate() in a cell. There is no need to pass any parameters.

###### Data Description
A subset of TROIKA dataset was used to train and test the model. The training dataset consist of one channel of PPG signal and three axis accelerometer signals recorded  from  subjects  with  age  from  18  to  35. For  each  subject,  the  PPG  signals were  recorded  from  wrist  by  two  pulse  oximeters  with  green  LEDs  (wavelength:  515nm).  Their distance (from center to center) was 2 cm. The acceleration signal was also recorded from wrist by a three-axis  accelerometer.  Both  the  pulse  oximeter  and  the  accelerometer  were  embedded  in  a wristband.   All  signals  were  sampled  at  125  Hz. The ECG signal from this dataset was used as a reference.

The dataset has limitations as to get a better estimate, we can have a dataset which also has dtat from gyroscope and magnetometer. The gyroscope measures angular velocity while the magnetometer measures absolute orientation.

###### Algorithm Description and Performace

The algorithm uses the Power Spectral Density(PSD) estimates to select frequencies with high PSDs for both PPG and Accelerometer signals. It then remove the frequencies which could have been attributed by accelerometer by comaring the PPG frequency to that of top three frequencies of accelerometer. 

The algorithm uses PPG uses blood flow in our hands to estimate signals while the accelorometer measures accelerationsignals in three axis. 

The algorithm outputs absoluter error and confidence. The absolute error is calculate by using reference heart beats which are derived using ECG signals. The confidence is calculated by using signal to noise ratio. The pulse rate is estimated every 2 seconds.

The estimates can be distorted by noise from other factors like 
* Melanin
* Arm movement
* Arm position
* Finger movement

Only by accounting for above factors we can have a better estimate. In this project, we have we were able to nullify noise from accelrometer. So, the algorithm is limited in its application. We used MAE at 90% availability as our metric to measure performace. The algorithm may also fail to perform properly for datasets where there is a sudden change in heartbeats. The error rate would increase in such a scenario.


-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 